<a href="https://colab.research.google.com/github/NooriDan/ShiftScheduler/blob/main/OR-tools/Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TA Scheduling

## Imports

In [1]:
!pip install ortools

In [2]:
from dataclasses import dataclass, field
from typing import Annotated, List, Dict, Optional
from datetime import datetime, time, timedelta
import csv
import openpyxl
from pathlib import Path
import pandas as pd
import numpy as np
import os
from ortools.sat.python import cp_model
print("Importing was successful")


Importing was successful


## Naive Programming

In [ ]:
def schedule_tas_debug(availability, ta_requirements, shift_requirements):
    num_tas = len(availability)
    num_shifts = len(availability[0])

    print("Availability Matrix:")
    for i in range(num_tas):
        print(f"TA {i}: {availability[i]}")

    print("\nTA Requirements:", ta_requirements)
    print("Shift Requirements:", shift_requirements)

    # Create the CP-SAT model
    model = cp_model.CpModel()

    # Decision variables: x[i][j] is 1 if TA i is assigned to shift j, else 0
    x = {}
    for i in range(num_tas):
        for j in range(num_shifts):
            x[i, j] = model.NewBoolVar(f'x[{i}][{j}]')

    # Constraint: TAs must meet their shift requirements
    for i in range(num_tas):
        model.Add(sum(x[i, j] for j in range(num_shifts)) == ta_requirements[i])
        print(f"Added TA {i} shift requirement: {ta_requirements[i]}")

    # Constraint: Shifts must meet staffing requirements
    for j in range(num_shifts):
        model.Add(sum(x[i, j] for i in range(num_tas)) == shift_requirements[j])
        print(f"Added shift {j} staffing requirement: {shift_requirements[j]}")

    # Constraint: Respect TA availability
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == -1:  # Unavailable
                model.Add(x[i, j] == 0)
                print(f"TA {i} cannot work shift {j}")

    # Objective: Maximize TA satisfaction
    objective_terms = []
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == 1:  # Desired
                objective_terms.append(10 * x[i, j])  # High weight for desired
            elif availability[i][j] == 0:  # Undesired
                objective_terms.append(-1 * x[i, j])  # Low weight for undesired
    model.Maximize(sum(objective_terms))

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Extract the solution
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        schedule = {}
        for i in range(num_tas):
            schedule[i] = [j for j in range(num_shifts) if solver.Value(x[i, j]) == 1]
        return schedule
    else:
        print("\nNo feasible solution found. Possible reasons:")
        print("- Check if total TA shift requirements match total shift needs.")
        print("- Ensure there are enough available TAs for each shift.")
        print("- Check if constraints are overly strict.")
        return None

# Example Input
availability_as_array_int = [
    [1, 0, -1, 1, 1],   # TA 0: Desired, Undesired, Unavailable, Desired, Desired
    [1, 1, 0, -1, 1],   # TA 1: Desired, Desired, Undesired, Unavailable, Desired
    [-1, 1, 1, 0, 1],   # TA 2: Unavailable, Desired, Desired, Undesired, Desired
]
ta_requirements = [2, 3, 2]  # TA 0: 2 shifts, TA 1: 1 shift, TA 2: 2 shifts
shift_requirements = [2, 1, 1, 1, 2]  # Shifts: 2 TAs, 1 TA, 1 TA, 1 TA, 2 TAs

# Solve
solution = schedule_tas_debug(availability_as_array_int, ta_requirements, shift_requirements)
if solution:
    print("\nFeasible Solution:")
    for ta, shifts in solution.items():
        print(f"TA {ta} assigned to shifts: {shifts}")
else:
    print("No feasible solution found.")


## Class-based Implementation: Scheduler

### Data classes

In [19]:
@dataclass
class TA:
    id: int
    macid: str
    name: str
    req_shift_per_week: int
    availability_as_dict: Dict[str, str] = field(default_factory=dict)
    availability_as_array_int: List[int] = field(default_factory=list)
    # to be planned
    assigned_shifts: List[int] = field(default_factory=list)

@dataclass
class Shift:
    id: int
    name: str
    req_ta_per_shift: int
    series: str
    day_of_week: str
    date: datetime
    start_time: time
    duration: timedelta = timedelta(hours=3, minutes=0)
    # to be planned
    assigned_tas: List[TA] = field(default_factory=list)

    def __post_init__(self):
        # Combine date and start_time to create a datetime object
        start_datetime = datetime.combine(self.date, self.start_time)
        # Calculate end_datetime by adding duration
        end_datetime = start_datetime + self.duration
        # Extract the time component from end_datetime
        self.end_time = end_datetime.time()

@dataclass
class Schedule:
    tas: List[TA] = field(default_factory=list)
    shifts: List[Shift] = field(default_factory=list)

### Solver Logic

In [20]:
class TA_Scheduling:
    def __init__(self, schedule, enable_logs: bool = False):
        self.schedule = schedule
        # Solver configuration
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.solver.parameters.linearization_level = 0
        self.solver.parameters.num_search_workers  = os.cpu_count()//2 + 1
        self.solver.parameters.max_time_in_seconds = 5.0   # Limit to 5 seconds
        self.solver.parameters.log_search_progress = enable_logs  # Enable logs
        self.solver_status = None
        # Properties to be computed
        self.assigment_matrix = {}

    def add_ta(self, ta: TA):
        self.schedule.tas.append(ta)

    def add_shift(self, shift: Shift):
        self.schedule.shifts.append(shift)

    def update_availability(self, ta_id: str, availability_as_array_int: List[int]):
        ta: TA = next((ta for ta in self.schedule.tas if ta.id == ta_id), None)
        if ta:
            total_available_count = sum( 1 for x in availability_as_array_int if x!= -1 )
            if ta.req_shift_per_week > total_available_count:
                print(f"TA_{ta_id} {ta.name} is not available enough to meet their requirement\n{ta.req_shift_per_week} required\n{total_available_count} was given")
                return False
            ta.availability_as_array_int = availability_as_array_int
        else:
            print(f"TA with ID {ta_id} not found.")
            return False
        return True

    def update_shift_requirements(self, shift_id: str, req_ta_per_shift: int):
        shift = next((shift for shift in self.schedule.shifts if shift.id == shift_id), None)
        if shift:
            shift.req_ta_per_shift = req_ta_per_shift
        else:
            print(f"Shift with ID {shift_id} not found.")

    def update_ta_requirements(self, ta_id: str, req_shift_per_week: int):
        ta = next((ta for ta in self.schedule.tas if ta.id == ta_id), None)
        if ta:
            ta.req_shift_per_week = req_shift_per_week
        else:
            print(f"TA with ID {ta_id} not found.")

    # 1 - Create Desicion Variables
    def create_decision_variables(self):
        for ta in self.schedule.tas:
            for shift in self.schedule.shifts:
                self.assigment_matrix[(ta.id, shift.id)] = self.model.NewBoolVar(f'availability_{ta.id}_{shift.id}')

    # 2 - Constraint Definitions
    # (2.1) TAs must meet their shift requirements
    def tas_meet_shift_requirements(self):
        for ta in self.schedule.tas:
            self.model.Add(sum(self.assigment_matrix[(ta.id, shift.id)] for shift in self.schedule.shifts) >= ta.req_shift_per_week)

    # (2.2) Shifts must meet staffing requirements
    def shifts_meet_staffing_requirements(self):
        for shift in self.schedule.shifts:
            self.model.Add(sum(self.assigment_matrix[(ta.id, shift.id)] for ta in self.schedule.tas) == shift.req_ta_per_shift)

    # (2.3) Respect TA availability
    def respect_ta_availability(self):
      for ta in self.schedule.tas:
        for shift in self.schedule.shifts:
          if ta.availability_as_array_int[shift.id] == -1:  # Unavailable
            self.model.Add(self.assigment_matrix[(ta.id, shift.id)] == 0)

    # 3 - Define Objective function
    def objective_function(self):
        objective_terms = []
        for ta in self.schedule.tas:
            for shift in self.schedule.shifts:
                if ta.availability_as_array_int[shift.id] == 1:
                    objective_terms.append(10 * self.assigment_matrix[(ta.id, shift.id)])
                elif ta.availability_as_array_int[shift.id] == 0:
                    objective_terms.append(-1 * self.assigment_matrix[(ta.id, shift.id)])
        self.model.Maximize(sum(objective_terms))

    # Put everything together
    def solve(self):
        self.create_decision_variables()
        # Apply constraints
        self.tas_meet_shift_requirements()
        self.shifts_meet_staffing_requirements()
        self.respect_ta_availability()
        # Optional Objective function
        self.objective_function()

        self.status = self.solver.Solve(self.model)

        if self.status == cp_model.OPTIMAL or self.status == cp_model.FEASIBLE:
            print("\nFeasible Solution:")
            for ta in self.schedule.tas:
                ta_shifts_str = [f"{shift.series}_{shift.id}" for shift in self.schedule.shifts if self.solver.Value(self.assigment_matrix[(ta.id, shift.id)]) == 1]
                print(f"TA {ta.id} requires {ta.req_shift_per_week} given {len(ta_shifts_str)}: {ta_shifts_str}")
                # Update TA and Shift objects
                for shift in self.schedule.shifts:
                    if self.solver.Value(self.assigment_matrix[(ta.id, shift.id)]) == 1:
                        ta.assigned_shifts.append(shift)
                        shift.assigned_tas.append(ta)
        else:
            print("\nNo feasible solution found. Possible reasons:")
            print("- Check if total TA shift requirements match total shift needs.")
            print("- Ensure there are enough available TAs for each shift.")
            print("- Check if constraints are overly strict.")

        return self.status


### Printing & Reporting

In [21]:
class Schedule_Utils:
    def __init__(self, output_dir: str = "outputs"):
        self.output_dir = output_dir

    # Reporting Utils
    def group_shifts_by_day_of_week(self, schedule_solution: TA_Scheduling):
        workday_shifts = {}
        for shift in schedule_solution.schedule.shifts:
            if shift.day_of_week not in workday_shifts:
                workday_shifts[shift.day_of_week] = []
            workday_shifts[shift.day_of_week].append(shift)

        # Sort shifts for each workday based on start_time
        for day in workday_shifts:
            workday_shifts[day] = sorted(workday_shifts[day], key=lambda s: s.start_time)
        return workday_shifts

    def report(self, schedule_solution: TA_Scheduling, filename='ta_schedule_report.csv'):
        # Step 1: Group shifts by day_of_week and sort them by start_time
        filename = f"{self.output_dir}/{filename}"
        workday_shifts = self.group_shifts_by_day_of_week(schedule_solution)

        # Step 2: Prepare the header for the CSV file
        days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        header = ['Shift/TA'] + days_of_week  # First column will be shift names, rest are days

        # Step 3: Determine the max number of rows across all workdays
        max_rows = max(len(shifts) for shifts in workday_shifts.values())  # Find the max number of shifts per day

        # Step 4: Open the CSV file for writing
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)

            # Step 5: Write the header row to the CSV
            writer.writerow(header)

            # Step 6: Write the shift rows
            for row_index in range(max_rows):
                row = ['']  # The first column is for shift names (e.g., Lab_0_0)
                for day in days_of_week:
                    shifts = workday_shifts.get(day, [])
                    if row_index < len(shifts):
                        shift = shifts[row_index]
                        # Collect assigned TAs for this shift
                        assigned_tas = [f"{ta.id}_{ta.name}" for ta in shift.assigned_tas]
                        # Prepare the cell content with shift name and assigned TAs
                        cell_content = f"{shift.series}_{shift.id}\n" + "\n".join(assigned_tas) if assigned_tas else f"{shift.series}_{shift.id}\nNo TAs assigned"
                    else:
                        cell_content = ""  # Empty cell if no shift for this row
                    row.append(cell_content)

                # Step 7: Write the row to the CSV
                writer.writerow(row)

        print(f"Report has been saved to {filename}")

    def report_timeseries(self, schedule_solution: TA_Scheduling, filename='ta_schedule_report_timeseries.csv'):
        filename = f"{self.output_dir}/{filename}"
        # Step 1: Extract and sort the shifts based on start time
        shifts_sorted_by_start_time = sorted(schedule_solution.schedule.shifts, key=lambda s: s.start_time)

        # Step 2: Prepare the header for the CSV file using the start time of each shift
        header = ['TA Name/ID']  # First column will be TA names and IDs
        for shift in shifts_sorted_by_start_time:
            # Format date and time as a string (assumes shift has a start_time field)
            date_combined  = datetime.combine(shift.date, shift.start_time)
            start_time_str = date_combined.strftime('%Y-%m-%d %H:%M')
            header.append(start_time_str)

        # Step 3: Determine the max number of rows (based on the most TAs assigned to a shift)
        max_rows = max(len(shift.assigned_tas) for shift in schedule_solution.schedule.shifts)  # Find max number of TAs per shift

        # Step 4: Open the CSV file for writing
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)

            # Step 5: Write the header row to the CSV
            writer.writerow(header)

            # Step 6: Write the rows for assigned TAs
            for row_index in range(max_rows):
                row = ['']  # The first column is for TA names and IDs
                for shift in shifts_sorted_by_start_time:
                    # Collect assigned TAs for this shift
                    assigned_tas = [f"{ta.id} - {ta.name}" for ta in shift.assigned_tas]

                    # Prepare the cell content with TA name and ID, if available
                    if row_index < len(assigned_tas):
                        # Only show one TA per row in the corresponding shift column
                        cell_content = assigned_tas[row_index]
                    else:
                        cell_content = ""  # Empty cell if no TA is assigned to this row for the shift
                    row.append(cell_content)

                # Step 7: Write the row to the CSV
                writer.writerow(row)

        print(f"Report has been saved to {filename}")


### Testing

In [26]:
# Testing the class definitions above

shift1 = Shift(id= 0, name='Morning Shift',    series='L01', day_of_week='Monday',    date=datetime(2024, 12, 12), start_time=time(14, 30), req_ta_per_shift = 2)
shift2 = Shift(id= 1, name='Afternoon Shift',  series='L02', day_of_week='Tuesday',   date=datetime(2024, 12, 13), start_time=time(14, 30), req_ta_per_shift = 1)
shift3 = Shift(id= 2, name='Evening Shift',    series='L03', day_of_week='Wednesday', date=datetime(2024, 12, 14), start_time=time(18, 30), req_ta_per_shift = 2)
shift4 = Shift(id= 3, name='Night Shift',      series='L04', day_of_week='Thursday',  date=datetime(2024, 12, 15), start_time=time(18, 30), req_ta_per_shift = 1)
shift5 = Shift(id= 4, name='Late Night Shift', series='L05', day_of_week='Friday',    date=datetime(2024, 12, 16), start_time=time(18, 30), req_ta_per_shift = 2)

ta1 = TA(id= 0, macid = "aaa", name='A Doe',       req_shift_per_week=1)
ta2 = TA(id= 1, macid = "bbb", name='B Smith',     req_shift_per_week=2)
ta3 = TA(id= 2, macid = "ccc", name='C Smith', req_shift_per_week=1)
ta4 = TA(id= 3, macid = "ddd", name='D Brown',      req_shift_per_week=2)

schedule = Schedule()
schedule.tas = [ta1, ta2, ta3, ta4]
schedule.shifts = [shift1, shift2, shift3, shift4, shift5]

ta_scheduling = TA_Scheduling(schedule)

In [27]:
ta_scheduling.update_availability(ta_id= 0, availability_as_array_int=[1, 1, 0, 1,  -1])
ta_scheduling.update_availability(ta_id= 1, availability_as_array_int=[1, 0, 0, 0,  1])
ta_scheduling.update_availability(ta_id= 2, availability_as_array_int=[1, 0, 0, 0,  1])
ta_scheduling.update_availability(ta_id= 3, availability_as_array_int=[1, 0, 0, 1, -1])

True

In [28]:
status = ta_scheduling.solve()


Feasible Solution:
TA 0 requires 1 given 3: ['L01_0', 'L02_1', 'L03_2']
TA 1 requires 2 given 2: ['L03_2', 'L05_4']
TA 2 requires 1 given 1: ['L05_4']
TA 3 requires 2 given 2: ['L01_0', 'L04_3']


In [29]:
util = Schedule_Utils()
util.report(ta_scheduling)

Report has been saved to outputs/ta_schedule_report.csv


In [30]:
util.report_timeseries(ta_scheduling)

Report has been saved to outputs/ta_schedule_report_timeseries.csv


## Class-based Data Constructors
Constructs TA, and Shift objects from CSV files.

In [62]:
# TODO
class DataConstructor:
    def __init__(self, ta_csv_path, shift_csv_path, availability_folder, load=True):
        self.ta_csv_path = ta_csv_path
        self.shift_csv_path = shift_csv_path
        self.availability_folder = availability_folder
        # Data holders
        self.ta_data    = None  # Placeholder for TA data    (pandas frame)
        self.shift_data = None  # Placeholder for Shift data (pandas frame)
        self.schedule   = None
        self.avialability_matrix: Dict[str, Dict[int, int]] = {}
        if load:
            self.create()

    def create(self):
        # Construction routine
        self.schedule = Schedule()
        self.load_data()
        self.create_ta_objects()
        self.create_shift_objects()
        self.extract_availabilities()
        if (self.validate_ta_availability()):
            self.update_ta_availability()
            print(f"Data succussfully extracted :)")
        else: 
            print("\n========================================")
        return self.schedule
    
    def extract_availabilities(self):
        print(self.availability_folder)
        filenames = self._find_xlsx_files(self.availability_folder)

        for file in filenames:
            mac_id, availability = self._extract_availability_from_xlsx(file)
            self.avialability_matrix[mac_id] = availability
        
        return self.avialability_matrix
    
    def validate_ta_availability(self, log_error=True):
        missing_ta = 0
        for ta in self.schedule.tas:
            if self.avialability_matrix.get(ta.macid) is None:
                missing_ta += 1
                if log_error:
                    print("========================================")
                    print(f"Found no record for TA: {ta.name}")
                    print(f"MacID = {ta.macid}")
                    print(f"missing {missing_ta}/{len(self.schedule.tas)} forms so far")
                    print("========================================")
            else:
                count_available = sum(1 for availability in self.avialability_matrix[ta.macid] if availability != "Unavailable")
                if count_available < ta.req_shift_per_week:
                    if log_error:
                        print(f"TA {ta.id} has insufficient availability.")
                        print(f"Available shifts: {count_available}")
                        print(f"Required shifts: {ta.req_shift_per_week}")

                    return False
        return not missing_ta
    
    def update_ta_availability(self):
        mapping: Dict[str, int] = {
            "Unavailable" : -1,
            "Desired"     : 1,
            "Undesired"   : 0
        }
        if (self.validate_ta_availability(log_error=False)):
            for ta in self.schedule.tas:
                ta.availability_as_dict = self.avialability_matrix.get(ta.macid)
                for shift in self.schedule.shifts:
                    ta.availability_as_array_int.append(mapping.get(ta.availability_as_dict.get(shift.series)))
            return True
        return False
    
    def _extract_availability_from_xlsx(self, file_path: str, availability_tbl_name: str = "availability_tbl"):
        # Load the workbook and select the sheet
        wb = openpyxl.load_workbook(file_path)
        ws = wb['availability']
        
        # Extracting TA details from the top of the sheet
        mac_id = ws['B1'].value
        ta_name = ws['B2'].value
        ta_id   = ws['B3'].value

        # # Print table names in the sheet
        # print("Tables in the sheet:", ws.tables.keys())

        # Check if availability_tbl_name exists
        if availability_tbl_name not in ws.tables:
            print("Table 'availability_tbl' not found.")

        tbl = ws.tables[availability_tbl_name]
        
        # Extract the table range
        table_range = tbl.ref  # e.g., "A5:C10"
        # print(f"Table '{availability_tbl_name}' range: {table_range}")

        # Extract the rows from the table range
        rows = ws[table_range]

        # Convert rows into a list of lists
        data = [[cell.value for cell in row] for row in rows]

        # Convert the data into a pandas DataFrame
        headers = data[0]       # First row contains headers
        table_df = pd.DataFrame(data[1:], columns=headers)  # Remaining rows are data
        
        availability = {}
        for index, row in table_df.iterrows():
            shift = row['Shift Series']
            shift_availability = row['Availability']
            availability[shift] = shift_availability   

        # print(availability)
        return mac_id, availability

    def _find_csv_files(self, directory):
        path = Path(directory)
        return [str(file) for file in path.glob("*.csv") if not file.name.startswith('~$')]  # Exclude files starting with ~$ and no recursive search

    def _find_xlsx_files(self, directory):
        path = Path(directory)
        return [str(file) for file in path.glob("*.xlsx") if not file.name.startswith('~$')]  # Exclude files starting with ~$ and no recursive search

    def load_data(self):
        self.ta_data    = pd.read_csv(self.ta_csv_path)
        self.shift_data = pd.read_csv(self.shift_csv_path)

    def create_ta_objects(self):
        for index, row in self.ta_data.iterrows():
            ta = TA(
                id= index,
                macid=row['macid'],
                name=row['name'],
                req_shift_per_week=row['req_shift_per_week']
            )
            self.schedule.tas.append(ta)

    def create_shift_objects(self):
        for index, row in self.shift_data.iterrows():
            shift = Shift(
                id = index,
                name = row['name'],
                series = row['series'],
                day_of_week = row['day_of_week'],
                date = datetime.strptime(row['date'], '%Y-%m-%d').date(),
                start_time = datetime.strptime(row['start_time'], '%H:%M').time(),
                req_ta_per_shift = row['req_ta_per_shift']
            )
            self.schedule.shifts.append(shift)


### Testing

In [63]:
constructor = DataConstructor(ta_csv_path="data/ta_list.csv",
                shift_csv_path="data/shift_list.csv",
                availability_folder="data/availability")

data/availability
Data succussfully extracted :)


In [64]:
constructor.schedule.shifts

[Shift(id=0, name='Lab 1', req_ta_per_shift=2, series='L01', day_of_week='Mon', date=datetime.date(2024, 1, 7), start_time=datetime.time(14, 30), duration=datetime.timedelta(seconds=10800), assigned_tas=[]),
 Shift(id=1, name='Lab 1', req_ta_per_shift=3, series='L02', day_of_week='Mon', date=datetime.date(2024, 1, 7), start_time=datetime.time(18, 30), duration=datetime.timedelta(seconds=10800), assigned_tas=[]),
 Shift(id=2, name='Lab 1', req_ta_per_shift=2, series='L03', day_of_week='Tue', date=datetime.date(2024, 1, 8), start_time=datetime.time(14, 30), duration=datetime.timedelta(seconds=10800), assigned_tas=[]),
 Shift(id=3, name='Lab 1', req_ta_per_shift=3, series='L04', day_of_week='Tue', date=datetime.date(2024, 1, 8), start_time=datetime.time(18, 30), duration=datetime.timedelta(seconds=10800), assigned_tas=[]),
 Shift(id=4, name='Lab 1', req_ta_per_shift=2, series='L05', day_of_week='Wed', date=datetime.date(2024, 1, 9), start_time=datetime.time(14, 30), duration=datetime.time

In [65]:
constructor.schedule.tas

[TA(id=0, macid='noorizad', name='Danial Noori Zadeh', req_shift_per_week=2, availability_as_dict={'L01': 'Desired', 'L02': 'Undesired', 'L03': 'Desired', 'L04': 'Desired', 'L05': 'Undesired', 'L06': 'Desired', 'L07': 'Desired', 'L08': 'Undesired', 'L09': 'Undesired', 'L10': 'Unavailable'}, availability_as_array_int=[1, 0, 1, 1, 0, 1, 1, 0, 0, -1], assigned_shifts=[]),
 TA(id=1, macid='roghanim', name='Moein Roghani', req_shift_per_week=1, availability_as_dict={'L01': 'Undesired', 'L02': 'Unavailable', 'L03': 'Desired', 'L04': 'Desired', 'L05': 'Desired', 'L06': 'Desired', 'L07': 'Desired', 'L08': 'Undesired', 'L09': 'Undesired', 'L10': 'Unavailable'}, availability_as_array_int=[0, -1, 1, 1, 1, 1, 1, 0, 0, -1], assigned_shifts=[]),
 TA(id=2, macid='roghania', name='Ali Roghani', req_shift_per_week=2, availability_as_dict={'L01': 'Undesired', 'L02': 'Undesired', 'L03': 'Undesired', 'L04': 'Undesired', 'L05': 'Undesired', 'L06': 'Desired', 'L07': 'Desired', 'L08': 'Undesired', 'L09': 'De

In [66]:
constructor.avialability_matrix

{'roghanim': {'L01': 'Undesired',
  'L02': 'Unavailable',
  'L03': 'Desired',
  'L04': 'Desired',
  'L05': 'Desired',
  'L06': 'Desired',
  'L07': 'Desired',
  'L08': 'Undesired',
  'L09': 'Undesired',
  'L10': 'Unavailable'},
 'baratia': {'L01': 'Undesired',
  'L02': 'Undesired',
  'L03': 'Undesired',
  'L04': 'Undesired',
  'L05': 'Undesired',
  'L06': 'Desired',
  'L07': 'Desired',
  'L08': 'Undesired',
  'L09': 'Desired',
  'L10': 'Desired'},
 'ebi': {'L01': 'Desired',
  'L02': 'Undesired',
  'L03': 'Undesired',
  'L04': 'Undesired',
  'L05': 'Unavailable',
  'L06': 'Unavailable',
  'L07': 'Desired',
  'L08': 'Undesired',
  'L09': 'Desired',
  'L10': 'Desired'},
 'saram': {'L01': 'Undesired',
  'L02': 'Desired',
  'L03': 'Desired',
  'L04': 'Undesired',
  'L05': 'Desired',
  'L06': 'Desired',
  'L07': 'Unavailable',
  'L08': 'Undesired',
  'L09': 'Desired',
  'L10': 'Undesired'},
 'noorizad': {'L01': 'Desired',
  'L02': 'Undesired',
  'L03': 'Desired',
  'L04': 'Desired',
  'L05': 

# General Optimization Practice

## CP-SAT - Nurse Scheduling

In [ ]:
from ortools.sat.python import cp_model

### Data Creation

In [ ]:
num_nurses = 4
num_shifts = 3
num_days = 3
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

### Create the model

In [ ]:
model = cp_model.CpModel()

### Create the variables
The array defines assignments for shifts to nurses as follows: shifts[(n, d, s)] equals 1 if shift s is assigned to nurse n on day d, and 0 otherwise.

In [ ]:
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")

### Code the constraints
Next, we show how to assign nurses to shifts subject to the following constraints:

Each shift is assigned to a single nurse per day.
Each nurse works at most one shift per day.

Here's the code that creates the first condition

In [ ]:
for d in all_days:
    for s in all_shifts:
        model.add_exactly_one(shifts[(n, d, s)] for n in all_nurses)

The last line says that for each shift, the sum of the nurses assigned to that shift is 1.

Next, here's the code that requires that each nurse works at most one shift per day.

In [ ]:
for n in all_nurses:
    for d in all_days:
        model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

For each nurse, the sum of shifts assigned to that nurse is at most 1 ("at most" because a nurse might have the day off).

**Assign shifts evenly**

In [ ]:
# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            shifts_worked.append(shifts[(n, d, s)])
    model.add(min_shifts_per_nurse <= sum(shifts_worked))
    model.add(sum(shifts_worked) <= max_shifts_per_nurse)

Since there are num_shifts * num_days total shifts in the schedule period, you can assign at least (num_shifts * num_days) // num_nurses

shifts to each nurse, but some shifts may be left over. (Here // is the Python integer division operator, which returns the floor of the usual quotient.)

For the given values of num_nurses = 4, num_shifts = 3, and num_days = 3, the expression min_shifts_per_nurse has the value (3 * 3 // 4) = 2, so you can assign at least two shifts to each nurse. This is specified by the constraint (here in Python)

In [ ]:
model.add(min_shifts_per_nurse <= sum(shifts_worked))

Since there are nine total shifts over the three-day period, there is one remaining shift after assigning two shifts to each nurse. The extra shift can be assigned to any nurse.

The final line (here in Python)

In [ ]:
model.add(sum(shifts_worked) <= max_shifts_per_nurse)

### Update solver parameters

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

### Register a callback function
You need to register a callback on the solver that will be called at each solution. (kind of optional)

In [ ]:
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for d in range(self._num_days):
            print(f"Day {d}")
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.value(self._shifts[(n, d, s)]):
                        is_working = True
                        print(f"  Nurse {n} works shift {s}")
                if not is_working:
                    print(f"  Nurse {n} does not work")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = NursesPartialSolutionPrinter(
    shifts, num_nurses, num_days, num_shifts, solution_limit
)

### Invoke the solver

In [ ]:
solver.solve(model, solution_printer)

### Entire nurse scheduling program

In [ ]:
"""Example of a simple nurse scheduling problem."""
from ortools.sat.python import cp_model


def main() -> None:
    # Data.
    num_nurses = 4
    num_shifts = 3
    num_days = 3
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d, s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")

    # Each shift is assigned to exactly one nurse in the schedule period.
    for d in all_days:
        for s in all_shifts:
            model.add_exactly_one(shifts[(n, d, s)] for n in all_nurses)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

    # Try to distribute the shifts evenly, so that each nurse works
    # min_shifts_per_nurse shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of nurses, some nurses will
    # be assigned one more shift.
    min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    if num_shifts * num_days % num_nurses == 0:
        max_shifts_per_nurse = min_shifts_per_nurse
    else:
        max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        shifts_worked = []
        for d in all_days:
            for s in all_shifts:
                shifts_worked.append(shifts[(n, d, s)])
        model.add(min_shifts_per_nurse <= sum(shifts_worked))
        model.add(sum(shifts_worked) <= max_shifts_per_nurse)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True

    class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_nurses = num_nurses
            self._num_days = num_days
            self._num_shifts = num_shifts
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print(f"Solution {self._solution_count}")
            for d in range(self._num_days):
                print(f"Day {d}")
                for n in range(self._num_nurses):
                    is_working = False
                    for s in range(self._num_shifts):
                        if self.value(self._shifts[(n, d, s)]):
                            is_working = True
                            print(f"  Nurse {n} works shift {s}")
                    if not is_working:
                        print(f"  Nurse {n} does not work")
            if self._solution_count >= self._solution_limit:
                print(f"Stop search after {self._solution_limit} solutions")
                self.stop_search()

        def solutionCount(self):
            return self._solution_count

    # Display the first five solutions.
    solution_limit = 5
    solution_printer = NursesPartialSolutionPrinter(
        shifts, num_nurses, num_days, num_shifts, solution_limit
    )

    solver.solve(model, solution_printer)

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")
    print(f"  - solutions found: {solution_printer.solutionCount()}")


if __name__ == "__main__":
    main()